# **Machine Learning / 2024. március 5-e kedd / Király Márk (AX83OL)**

#Házi feladat
A Melbourne dataset felhasználásával készítsen egy lineáris regresszort a lakásárak (Price) meghatározására, úgy hogy a hiányzó adatokat (numerikus és kategórikus) imputálással pótolja (stratégiát Ön választ, adatot ne dobjon el). A modell építés során numerikus és kategórikus adatokat is használja fel! Az adatokat ossza fel 80% tréning, 20% validációs részre az imputálás után. A modellt értékelje ki MAE, R2 segítségével.

Folyamatok:
- adatok betöltése
- adatok felosztása tréning, teszt DataFramekre
- adatok (tréning, teszt) szétválasztása numerikus és kategórikus részekre
- adatok imputálása
- a kategórikus adatok kódolása
- numerikus és kategórikus adatok összefűzése (pd.concat([num_X_train, OH_cols_train], axis=1))
- modell készítése
- kiértékelés




# Beimportáljuk a megfelelő modulokat.

In [140]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Betöltjük és felosztjuk az adathalmazt.

In [141]:
# Load the dataset
url = "https://raw.githubusercontent.com/karsarobert/Machine_Learning_2024/main/melb_data.csv"
data = pd.read_csv(url)

# Split data into features and target
X = data.drop('Price', axis=1)
y = data['Price']

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396 entries, 0 to 18395
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     18396 non-null  int64  
 1   Suburb         18396 non-null  object 
 2   Address        18396 non-null  object 
 3   Rooms          18396 non-null  int64  
 4   Type           18396 non-null  object 
 5   Price          18396 non-null  float64
 6   Method         18396 non-null  object 
 7   SellerG        18396 non-null  object 
 8   Date           18396 non-null  object 
 9   Distance       18395 non-null  float64
 10  Postcode       18395 non-null  float64
 11  Bedroom2       14927 non-null  float64
 12  Bathroom       14925 non-null  float64
 13  Car            14820 non-null  float64
 14  Landsize       13603 non-null  float64
 15  BuildingArea   7762 non-null   float64
 16  YearBuilt      8958 non-null   float64
 17  CouncilArea    12233 non-null  object 
 18  Lattit

# Szétvágjuk az adathalmazt tréning és teszt halmazokra 80% - 20% arányban.

In [142]:
# Split data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

sets = {
    'X_train': X_train,
    'X_valid': X_valid,
    'y_train': y_train,
    'y_valid': y_valid
}
for setName, setValue in zip(sets.keys(), sets.values()):
    print(f'{setName}: {len(setValue)}')

X_train: 14716
X_valid: 3680
y_train: 14716
y_valid: 3680


# Kiválogatjuk a numerikus és kategorikus oszlopokat, majd a többit eldobjuk.

In [143]:
# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
print("Numerikus oszlopok: ",numerical_cols)

# Select categorical columns with relatively low cardinality (arbitrarily set at 10)
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]
print("Kategorikus oszlopok: ",categorical_cols)

# Keep selected columns only
my_cols = numerical_cols + categorical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

# Check the dataset length was not changed
print("Ellenőrizzük az elemszám változatlanságát: ",len(X_train),"és",len(X_valid))

# for Example
X_train

Numerikus oszlopok:  ['Unnamed: 0', 'Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
Kategorikus oszlopok:  ['Type', 'Method', 'Regionname']
Ellenőrizzük az elemszám változatlanságát:  14716 és 3680


,Unnamed: 0,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Type,Method,Regionname
2573,3349,4,7.8,3058.0,4.0,2.0,1.0,381.0,NaN,1938.0,-37.73370,144.95480,11204.0,h,SP,Northern Metropolitan
2091,2686,3,7.8,3124.0,3.0,1.0,1.0,544.0,160.0,1930.0,-37.84360,145.05810,8920.0,h,SP,Southern Metropolitan
4683,6065,2,5.6,3101.0,2.0,1.0,1.0,121.0,NaN,NaN,-37.81260,145.05340,10331.0,u,S,Southern Metropolitan
8832,11346,3,7.5,3123.0,3.0,2.0,2.0,200.0,NaN,NaN,-37.83960,145.05140,6482.0,h,VB,Southern Metropolitan
10469,13474,2,4.5,3181.0,2.0,1.0,1.0,2842.0,84.0,1920.0,-37.85130,144.99430,7717.0,u,S,Southern Metropolitan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9225,11849,2,13.9,3020.0,3.0,1.0,1.0,199.0,NaN,NaN,-37.77610,144.81640,2185.0,u,S,Western Metropolitan
13123,16889,2,7.2,3184.0,2.0,1.0,1.0,0.0,NaN,NaN,-37.88945,144.99015,8989.0,u,S,Southern Metropolitan
9845,12649,3,7.8,3058.0,3.0,2.0,2.0,500.0,148.0,1900.0,-37.73820,144.96530,11204.0,h,S,Northern Metropolitan
10799,13887,3,8.2,3012.0,3.0,1.0,0.0,370.0,NaN,NaN,-37.79160,144.87150,5058.0,h,PI,Western Metropolitan


# Gyors ellenőrzés a kategorikus változókat illetően.

In [144]:
# Categorical variables and their unique values
for cname in X_train.columns:
   if X_train[cname].dtype == "object" :
     print(str(cname) +': '+ str(X_train[cname].nunique()))

Type: 3
Method: 5
Regionname: 8


# Imputálás és one-hot encoding.

In [145]:
# Preprocessing for numerical data: imputation
numerical_transformer = SimpleImputer(strategy='mean') # mediánnal való pótlás

# Preprocessing for categorical data: imputation and one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # leggyakoribb értékekkel való pótlás
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, numerical_cols),
        ('categorical', categorical_transformer, categorical_cols)
    ])

# A model létrehozása és edzése.

In [146]:
# Define the model
model = LinearRegression()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

pipeline.fit(X_train, y_train)

# Predictions on training set
train_preds = pipeline.predict(X_train)

# Predictions on validation set
val_preds = pipeline.predict(X_valid)

# A model kiértékelése.

In [147]:
# Evaluate the model
mae = mean_absolute_error(y_valid, val_preds)
r2 = r2_score(y_valid, val_preds)

print('Mean Absolute Error:', round(mae,2))
print('R^2 Score:', r2)

Mean Absolute Error: 276683.28
R^2 Score: 0.5851344087268027
